In [1]:
import cupy as cp

def ipc_roundtrip_demo():
    # 1) Allocate device memory normally
    a = cp.arange(10, dtype=cp.int32)
    print("a (orig):", a)

    # 2) Export IPC handle from the device pointer
    #    a.data.ptr is the raw device pointer (int)
    handle = cp.cuda.runtime.ipcGetMemHandle(a.data.ptr)

    # 3) Import/open the handle -> returns a *device pointer* (int)
    #    Flags: cudaIpcMemLazyEnablePeerAccess is usually fine
    ptr2 = cp.cuda.runtime.ipcOpenMemHandle(
        handle,
        cp.cuda.runtime.ipcMemLazyEnablePeerAccess
    )

    try:
        # 4) Wrap ptr2 as a CuPy ndarray view.
        #    This is "unowned" memory: we must ensure 'a' stays alive.
        mem = cp.cuda.UnownedMemory(
            ptr2,
            a.nbytes,
            owner=a,                 # keep original allocation alive
            device_id=a.device.id
        )
        mp = cp.cuda.MemoryPointer(mem, 0)
        b = cp.ndarray(a.shape, dtype=a.dtype, memptr=mp)

        print("b (ipc view):", b)

        # 5) Prove it's the same underlying memory
        b[:] = b[:] * 10
        print("a after modifying b:", a)
        print("b after modify:", b)

        # Another quick check
        assert int(a[3].get()) == int((3 * 10))
        print("OK: b writes affected a")
    finally:
        # 6) Close the opened mapping.
        #    After this, b/mem are invalid to use.
        cp.cuda.runtime.ipcCloseMemHandle(ptr2)

ipc_roundtrip_demo()

a (orig): [0 1 2 3 4 5 6 7 8 9]


AttributeError: module 'cupy_backends.cuda.api.runtime' has no attribute 'ipcMemLazyEnablePeerAccess'